## 関数テスト用のnotebook

In [1]:
#データの読み込みから実行まで全てを実行するくRunnerクラスを定義する
from typing import Any, Dict, Tuple, Union
import sys
import numpy as np
import joblib
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.callbacks import Callback,ModelCheckpoint
from tensorflow.keras.layers import Dense,BatchNormalization,Activation
from tensorflow.keras.activations import relu

sys.path.append('../../')
from Utils.utils import *
from Utils.get_q import *

layer_name_list=['batch_normalization1', 'batch_normalization2', 'batch_normalization3','batch_normalization4', 'batch_normalization5',
                  'batch_normalization6', 'batch_normalization7', 'batch_normalization8','batch_normalization9','batch_normalization10',]

CFG = {
  'N': 100,
  'M': 6000,
  'L': 10,
  'ini_type' : 'A',
  'data_name': 'MNIST',
  'C' :10,


  'data_seed': 4244, #データのサンプリングを固定するためのseed
  'seed1' : 820,
  'seed2' : 314,
  'seed3' : 1228,
}


def preprocessing(X_train, X_test,y_train, y_test) -> np.ndarray:  
        X_train = X_train.reshape(X_train.shape[0], -1)
        X_test = X_test.reshape(X_test.shape[0], -1)

        X_train = X_train / 255
        X_test = X_test / 255
        # Use 32-bit instead of 64-bit float
        X_train = X_train.astype("float32")
        X_test = X_test.astype("float32")
        
        idx = np.random.choice(X_train.shape[0], size=6000)
        X_train = X_train[idx]
        y_train = y_train[idx]
        
        X_train_out = X_train
        X_test_out = X_test
        y_train_out = y_train
        y_test_out = y_test
    
        return X_train_out, y_train_out, X_test_out, y_test_out


def PCA_SS(input_train,input_test):
        # Make an instance of the Model
        pca = PCA(n_components=CFG['N'])
        scaler = StandardScaler()

        train_img = pca.fit_transform(input_train)
        train_img =scaler.fit_transform(train_img)
        
        test_img = pca.transform(input_test)
        test_img =scaler.transform(test_img)
        return train_img, test_img, pca, scaler

params= {
  'N': 100,
  'num_classes':10,
  'num_layers':10,
  'activation':'relu',
  'activation_last':'softmax',
  'learning_rate': 0.1,
  'loss': 'sparse_categorical_crossentropy',
  'metric': 'accuracy',
  'random_seed': 777,
  'bias_initializer': tf.keras.initializers.Constant(0.1),
  'optimizer':'adam',
  'epochs':5,
  'batch_size':256
}

In [2]:
set_seed(CFG['data_seed'])
mnist = tf.keras.datasets.mnist
(X_train, y_train),(X_test, y_test) = mnist.load_data()
print('='*15+'Preprocess data'+'='*15)
X_train, y_train, X_test, y_test = preprocessing(X_train, X_test,y_train, y_test)
print('Train size:',X_train.shape, 'Test size:', y_train.shape)
X_train, X_test, pca1, scaler1 = PCA_SS(X_train, X_test)
print('Train size:',X_train.shape, 'Test size:', y_train.shape)

===============Preprocess data===============
Train size: (6000, 784) Test size: (6000,)
Train size: (6000, 100) Test size: (6000,)


In [3]:
set_seed(CFG['seed1'])
w_intializer1 = tf.keras.initializers.RandomNormal(mean=0, stddev=1)
bias_initializer1 = tf.keras.initializers.Constant(0.1)
model1 = create_fully_model(params=params,W=w_intializer1,B=bias_initializer1)
history1 = model1.fit(X_train, y_train,
                batch_size=params["batch_size"],
                epochs=params["epochs"],
                verbose=1,
                shuffle=True,
                validation_data=(X_test, y_test),
                callbacks=[LogEpochIntermediateCallcack(layer_name_list=layer_name_list,CFG=CFG,X_train=X_train,
                                                        path=f"../Test/Test_output/spinA_{CFG['data_name']}_ini{CFG['ini_type']}_M{CFG['M']}_L{CFG['L']}_seed{CFG['seed1']}.pkl")]
                )

Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2024-01-04 20:13:15.806145: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-01-04 20:13:15.806335: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


109/188 [================>.............] - ETA: 0s

2024-01-04 20:13:16.091406: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2024-01-04 20:13:16.136499: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


122/188 [==================>...........] - ETA: 0s

...

  1/188 [..............................] - ETA: 40s

2024-01-04 20:13:19.628032: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


  1/188 [..............................] - ETA: 40s

2024-01-04 20:13:20.648169: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


 64/188 [=========>....................] - ETA: 0s

In [30]:
if CFG['ini_type'] == 'A':
            print('='*15+'ini_type A'+'='*15)
            set_seed(CFG['seed2'])
            w_intializer2 = tf.keras.initializers.RandomNormal(mean=0, stddev=1)
            bias_initializer2 = tf.keras.initializers.Constant(0.1)
elif CFG['ini_type'] == 'B':
    print('='*15+'ini type B'+'='*15)
    w_intializer2 = w_intializer1
    bias_initializer2 = bias_initializer1
    set_seed(CFG['seed3'])

model2 = create_fully_model(params=params,W=w_intializer2,B=bias_initializer2)    
history2 = model2.fit(X_train, y_train,
        batch_size=params["batch_size"],
        epochs=params["epochs"],
        verbose=1,
        shuffle=True,
        validation_data=(X_test, y_test),
        callbacks=[LogEpochIntermediateCallcack(layer_name_list=self.layer_list,CFG=CFG,X_train=X_train,
                                                path=f"../Output/Spin/spinB_{CFG['data_name']}_ini{CFG['ini_type']}_M{CFG['M']}_L{CFG['L']}_seed{CFG['seed2']}.pkl")]
        )

In [32]:
loss1 = joblib.load('../Output/Loss/model001_iniA_M6000_L10_seed820.pkl')

## Maskのテスト

In [14]:
import tensorflow as tf
def create_sparse_model(params:dict, W, B, Mask_list,Const_list):
    model=Sequential(name='sparse_connect_model')
    model.add(Dense(params['N'], kernel_initializer=W,bias_initializer=B,input_shape=(params['N'],), name='Affine1',kernel_constraint=CustomConstraint(mask=Mask_list[0],const=Const_list[0])))
    model.add(BatchNormalization(name='batch_normalization1'))
    model.add(Activation('relu', name='activation1'))
    for i in range(1, params['num_layers']-1):
        model.add(Dense(params['N'], kernel_initializer=W,bias_initializer=B,name=f'Affine{i+1}',kernel_constraint=CustomConstraint(mask=Mask_list[i],const=Const_list[i])))
        model.add(BatchNormalization(name=f'batch_normalization{i+1}'))
        model.add(Activation(params['activation'],name=f'activation{i+1}'))
        
    model.add(Dense(params['num_classes'],kernel_initializer=W,bias_initializer=B,name='output',kernel_constraint=CustomConstraint(mask=Mask_list[params["num_layers"]-1],const=Const_list[params["num_layers"]-1])))
    model.add(BatchNormalization(name=f'batch_normalization{params["num_layers"]}'))
    model.add(Activation('softmax', name=f'activation{params["num_layers"]}'))
    model.compile(loss=params['loss'], optimizer=params['optimizer'],metrics=[params['metric']])
    return model



def create_dense_coupling_model(params:dict, W, B, Mask_list,Const_list):
    model=Sequential(name='sparse_connect_model')
    model.add(Dense(params['N'], kernel_initializer=W,bias_initializer=B,input_shape=(params['N'],), name='Affine1',kernel_constraint=DenseCoupleConstraint(mask=Mask_list[0])))
    model.add(BatchNormalization(name='batch_normalization1'))
    model.add(Activation('relu', name='activation1'))
    for i in range(1, params['num_layers']-1):
        model.add(Dense(params['N'], kernel_initializer=W,bias_initializer=B,name=f'Affine{i+1}',kernel_constraint=DenseCoupleConstraint(mask=Mask_list[i])))
        model.add(BatchNormalization(name=f'batch_normalization{i+1}'))
        model.add(Activation(params['activation'],name=f'activation{i+1}'))
        
    model.add(Dense(params['num_classes'],kernel_initializer=W,bias_initializer=B,name='output',kernel_constraint=DenseCoupleConstraint(mask=Mask_list[params["num_layers"]-1])))
    model.add(BatchNormalization(name=f'batch_normalization{params["num_layers"]}'))
    model.add(Activation('softmax', name=f'activation{params["num_layers"]}'))
    model.compile(loss=params['loss'], optimizer=params['optimizer'],metrics=[params['metric']])
    return model

class CustomConstraint(tf.keras.constraints.Constraint):
    def __init__(self, mask, const):
        self.mask = mask
        self.const = const

    def __call__(self, w):
        # マスク行列を使用して、指定された部分を0で固定する
        w.assign(tf.math.multiply(w, self.mask) + self.const)
        return w
    
class DenseCoupleConstraint(tf.keras.constraints.Constraint):
    def __init__(self, mask):
        self.mask = mask

    def __call__(self, w):
        # マスク行列を使用して、指定された部分を0で固定する
        w.assign(tf.math.multiply(w, self.mask))
        return w 
    
# Weight可視化用のcalback
class Layer1WeightsCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        layer1_weights = self.model.layers[0].get_weights()[0][:,1]
        print(f"Epoch {epoch + 1} - Layer 1 Weights:")
        print(layer1_weights)

def get_mask(shape, C, dtype=int):
    masks = np.zeros(shape)
    consts = np.random.normal(size=shape)
    for col in range(shape[1]):
        non_zero_indices = np.random.choice(shape[0], C, replace=False)
        masks[non_zero_indices, col] = 1
        consts[non_zero_indices, col] = 0
    masks = tf.constant(masks, dtype=tf.float32)
    consts = tf.constant(consts, dtype=tf.float32)
    return masks, consts



def get_const(shape, G, dtype=int):
    """
    G:値が固定された結合の数
    """
    masks = np.zeros(shape)
    #consts = np.random.normal(size=shape)
    consts = np.ones(shape)
    for col in range(shape[1]):
        non_zero_indices = np.random.choice(shape[0], 100-G, replace=False)
        masks[non_zero_indices, col] = 1
        consts[non_zero_indices, col] = 0
    masks = tf.constant(masks, dtype=tf.float32)
    consts = tf.constant(consts, dtype=tf.float32)
    return masks, consts


### 結合を切断したネットワーク(Dense coupling)

In [12]:
mask_list = []
const_list = []
for i in range(CFG['L']-1):
    mask,const = get_mask(shape=(100,100),C=10)
    mask_list.append(mask)
    const_list.append(const)
            
mask, const = get_mask(shape=(100,10),C=10)
mask_list.append(mask)
const_list.append(const)
    
set_seed(seed=2023)
w_intializer1 = tf.keras.initializers.RandomNormal(mean=0, stddev=1)
bias_initializer1 = tf.keras.initializers.Constant(0.1)
model1 = create_dense_coupling_model(params=params,W=w_intializer1,B=bias_initializer1,Mask_list=mask_list,Const_list=const_list)
# モデルの学習にコールバックを適用する
callbacks = [Layer1WeightsCallback()]

# モデルの学習
history1 = model1.fit(X_train, y_train,
                      batch_size=params["batch_size"],
                      epochs=params["epochs"],
                      verbose=1,
                      shuffle=True,
                      validation_data=(X_test, y_test),
                      callbacks=callbacks)  # コール

Epoch 1/5


2024-01-04 20:58:43.000175: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


24/24 [==============================] - ETA: 0s - loss: 2.6887 - accuracy: 0.0962

2024-01-04 20:58:46.578936: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


Epoch 1 - Layer 1 Weights:
[ 0.          0.73604816  0.          0.          0.         -0.66310406
 -0.          0.          0.         -0.         -0.         -0.
 -0.          0.          0.         -0.         -0.         -0.
  0.         -0.          0.         -0.          0.          0.
 -0.          0.          0.          0.         -0.         -0.
 -0.          0.         -0.          0.          3.5269356  -0.
 -0.         -0.          0.         -0.         -0.         -0.
 -0.          0.          0.         -0.          0.         -0.
  0.         -0.          0.         -0.          0.         -1.1498586
  0.         -0.         -0.          0.7824707   0.         -0.
  0.         -0.         -0.         -0.          0.         -0.10947092
 -0.          0.         -0.          0.09348246  0.          0.
  0.         -0.          0.         -0.         -0.         -0.
  0.         -0.40903792  1.1775571  -0.          0.          0.
  0.          0.         -0.         -0.

### 結合の値を定数で固定したネットワーク

In [15]:
mask_list = []
const_list = []
for i in range(CFG['L']-1):
    mask,const = get_const(shape=(100,100),G=30)
    mask_list.append(mask)
    const_list.append(const)
            
mask, const = get_const(shape=(100,10),G=30)
mask_list.append(mask)
const_list.append(const)


set_seed(seed=2023)
w_intializer1 = tf.keras.initializers.RandomNormal(mean=0, stddev=1)
bias_initializer1 = tf.keras.initializers.Constant(0.1)
model1 = create_sparse_model(params=params,W=w_intializer1,B=bias_initializer1,Mask_list=mask_list,Const_list=const_list)
# モデルの学習にコールバックを適用する
callbacks = [Layer1WeightsCallback()]

# モデルの学習
history1 = model1.fit(X_train, y_train,
                      batch_size=params["batch_size"],
                      epochs=params["epochs"],
                      verbose=1,
                      shuffle=True,
                      validation_data=(X_test, y_test),
                      callbacks=callbacks)  # コール

Epoch 1/5


2024-01-04 21:00:31.886798: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


24/24 [==============================] - ETA: 0s - loss: 2.3162 - accuracy: 0.1195

2024-01-04 21:00:35.995740: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


Epoch 1 - Layer 1 Weights:
[ 1.          1.         -0.5375939   1.6434128  -0.02116001  1.
  1.         -0.65865207  2.7686167  -0.76809657  1.12623    -0.09483519
  1.         -0.76976234  0.2204066   1.         -0.6050223   1.7445585
  1.          1.          1.          0.44414982  0.28459975  0.76946557
  0.01584099 -0.05745828 -1.1750039   1.         -1.3129959   0.51507413
 -0.53240407  0.8213414   0.79644173  0.14277817  3.5221689   0.58482116
 -0.6801129   1.          1.371349    1.          1.3787649  -0.22322291
  1.         -0.99948496  1.         -0.6266995   0.9669731   1.
  0.35833704 -1.1806353   0.56520146  0.2931701  -0.24738201 -1.161876
  2.0493717   2.6579537   1.          1.          0.02643626 -0.46119818
  0.67494285 -0.2333037   1.          0.95016485 -0.19801186 -0.11356721
  0.4095694   1.          1.27351     0.10264956  1.         -1.7119399
  1.         -1.5069758   1.          1.          0.6408424   1.
 -2.4000497   1.          1.1746814  -1.3074291   1.